# Installing Scikit Learn

In [4]:
!pip3 install -U scikit-learn scipy matplotlib

Requirement already up-to-date: scipy in c:\users\nilsvn\appdata\local\programs\python\python36-32\lib\site-packages (1.4.1)
Requirement already up-to-date: matplotlib in c:\users\nilsvn\appdata\local\programs\python\python36-32\lib\site-packages (3.2.1)
Could not build wheels for scipy, since package 'wheel' is not installed.
Could not build wheels for matplotlib, since package 'wheel' is not installed.
Could not build wheels for numpy, since package 'wheel' is not installed.
Could not build wheels for cycler, since package 'wheel' is not installed.
Could not build wheels for kiwisolver, since package 'wheel' is not installed.
Could not build wheels for python-dateutil, since package 'wheel' is not installed.
Could not build wheels for pyparsing, since package 'wheel' is not installed.
Could not build wheels for six, since package 'wheel' is not installed.


You should consider upgrading via the 'c:\users\nilsvn\appdata\local\programs\python\python36-32\python.exe -m pip install --upgrade pip' command.


In [34]:
import pandas as pd
from sklearn import svm
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
df = pd.read_excel (r'Path where the Excel file is stored\File name.xlsx')
print (df)

import generic data, import attack data, merge into one (randomly) with corresponding label array [0,0,1,0...]

Eventually when testing multiclass data (4 labels, normal + 3 attack types) will I be able to change the threshold?

.data = [x][y]

.target = [x] (zeroes and ones if binary, corr. with data)

.feature_names = [y] (time, user, etc)

.target_names = [2] (if binary, normal/suspicious) ['normal', 'suspicious']

In [35]:
iris= datasets.load_iris()

In [36]:
X = iris.data[:,2]

In [37]:
y = iris.target

In [38]:
def SVM(): #todo: threshold as param
    global X
    global y
    X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=4)
    X_train_mod = X_train.reshape(-1,1)
    X_test_mod = X_test.reshape(-1,1)
    y_train_mod = y_train.reshape(-1,1)
    y_test_mod = y_test.reshape(-1,1)
    model= svm.SVC(kernel='linear')
    model.fit(X_train_mod, y_train_mod)
    y_pred = model.predict(X_test_mod)
    return y_test_mod, y_pred

In [40]:
y_test, y_pred = SVM()
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.classification_report(y_test, y_pred, digits=3))

[[16  0  0]
 [ 0  4  1]
 [ 0  0  9]]
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        16
           1      1.000     0.800     0.889         5
           2      0.900     1.000     0.947         9

    accuracy                          0.967        30
   macro avg      0.967     0.933     0.945        30
weighted avg      0.970     0.967     0.966        30



c:\users\nilsvn\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
